étape_0_bis.py

In [ ]:
# Chemin du fichier d'entrée et de sortie
input_file = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/instructions_list.txt"
output_file = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/instructions_list_cleaned.txt"

# Lire le fichier et filtrer les lignes
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Filtrer les lignes ne commençant pas par un chiffre et ne finissant pas par "]"
cleaned_lines = [line for line in lines if not line.strip().startswith(tuple("0123456789")) and not line.strip().endswith("]")]

# Enregistrer le nouveau fichier
with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(cleaned_lines)

# Afficher le nombre de lignes restantes
print(f"Nombre de lignes dans le nouveau fichier : {len(cleaned_lines)}")
print(f"Fichier nettoyé enregistré dans : {output_file}")

étape_0_instructions.py

In [ ]:
import os

def extract_all_instructions(folder_path, output_file):
    instructions_set = set()
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Vérifie si c'est un fichier JSON
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    lines = f.readlines()  # Lire le fichier ligne par ligne
                    
                    for line in lines:
                        # Extraire le mot après le 2ème ":"
                        parts = line.split(":")
                        if len(parts) > 2:
                            word_after_second_colon = parts[2].strip().split()[0]  # Prendre le premier mot après le deuxième ":"
                            instructions_set.add(word_after_second_colon)  # Utiliser .add() pour un seul mot
                except Exception as e:
                    print(f"Erreur de lecture du fichier {filename} : {e}")

    # Trier la liste et l'enregistrer dans un fichier
    sorted_instructions = sorted(instructions_set)
    with open(output_file, "w", encoding="utf-8") as f_out:
        for instr in sorted_instructions:
            f_out.write(instr + "\n")
    
    print(f"Nombre total d'instructions uniques : {len(sorted_instructions)}")
    print(f"Liste des instructions enregistrée dans : {output_file}")

# Exemple d'utilisation
folder_path = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/folder_training_set"
output_file = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/instructions_list.txt"

extract_all_instructions(folder_path, output_file)

étape_1_création_features_train.py

In [ ]:
import os
import re
import pandas as pd

def create_features_dataframe(folder_path, instructions_list, output_csv):
    # Charger la liste des instructions depuis le fichier
    with open(instructions_list, "r", encoding="utf-8") as f:
        all_instructions = [line.strip() for line in f.readlines()]
    
    # Initialiser une liste pour stocker les features
    data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Vérifie si c'est un fichier JSON
            file_path = os.path.join(folder_path, filename)
            file_id = filename.replace(".json", "")  # Supprimer .json
            
            features = {instr: 0 for instr in all_instructions}  # Init à 0 pour toutes les instructions
            features["filename"] = file_id  # Ajouter le nom du fichier SANS .json
            
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    lines = f.readlines()  # Lire le fichier ligne par ligne
                    
                    for line in lines:
                        # Extraire le mot après le 2ème ":"
                        parts = line.split(":")
                        if len(parts) > 2:
                            word_after_second_colon = parts[2].strip().split()[0]  # Prendre le premier mot après le deuxième ":"
                            
                            if word_after_second_colon in features:  # Vérifier si l'instruction est dans la liste
                                features[word_after_second_colon] = 1  # Marquer sa présence
                except Exception as e:
                    print(f"Erreur de lecture du fichier {filename} : {e}")

            data.append(features)

    # Convertir en DataFrame
    df = pd.DataFrame(data)
    
    # Sauvegarder en CSV
    df.to_csv(output_csv, index=False)
    
    print(f"DataFrame enregistré : {output_csv}")
    print(df.head())  # Afficher un aperçu

# Exemple d'utilisation
folder_path = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/folder_training_set"
instructions_list = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/instructions_list_cleaned.txt"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_train.csv"

create_features_dataframe(folder_path, instructions_list, output_csv)


étape_2_bis_fusion_train.py

In [ ]:
import pandas as pd

def merge_features_with_labels(features_csv, labels_csv, output_csv):
    # Charger les fichiers CSV
    df_features = pd.read_csv(features_csv)
    df_labels = pd.read_csv(labels_csv)  # Vérifier le bon séparateur
    
    # Renommer la colonne `filename` du fichier features pour qu'elle corresponde à `name`
    df_features.rename(columns={"filename": "name"}, inplace=True)

    # Fusionner les deux DataFrames sur `filename`
    df_final = df_labels.merge(df_features, on="name", how="inner")

    # Sauvegarder le DataFrame fusionné
    df_final.to_csv(output_csv, index=False)

    print(f"DataFrame fusionné enregistré : {output_csv}")
    print(df_final.head())  # Afficher un aperçu

features_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_train_inst.csv"
labels_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/train_data.csv"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/new_train_data.csv"

merge_features_with_labels(features_csv, labels_csv, output_csv)


étape_2_fusion_train.py

In [ ]:
import pandas as pd

def merge_features_with_labels(features_csv, labels_csv, output_csv):
    # Charger les fichiers CSV
    df_features = pd.read_csv(features_csv)
    df_labels = pd.read_csv(labels_csv, sep=";")  # Vérifier le bon séparateur
    
    # Renommer la colonne `filename` du fichier features pour qu'elle corresponde à `name`
    df_features.rename(columns={"filename": "name"}, inplace=True)

    # Fusionner les deux DataFrames sur `filename`
    df_final = df_labels.merge(df_features, on="name", how="inner")

    # Sauvegarder le DataFrame fusionné
    df_final.to_csv(output_csv, index=False)

    print(f"DataFrame fusionné enregistré : {output_csv}")
    print(df_final.head())  # Afficher un aperçu

features_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_train.csv"
labels_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/training_set_metadata.csv"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/train_data.csv"

merge_features_with_labels(features_csv, labels_csv, output_csv)


étape_3_création_features_test.py

In [ ]:
import os
import re
import pandas as pd

def create_features_dataframe(folder_path, instructions_list, output_csv):
    # Charger la liste des instructions depuis le fichier
    with open(instructions_list, "r", encoding="utf-8") as f:
        all_instructions = [line.strip() for line in f.readlines()]
    
    # Initialiser une liste pour stocker les features
    data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Vérifie si c'est un fichier JSON
            file_path = os.path.join(folder_path, filename)
            file_id = filename.replace(".json", "")  # Supprimer .json
            
            features = {instr: 0 for instr in all_instructions}  # Init à 0 pour toutes les instructions
            features["filename"] = file_id  # Ajouter le nom du fichier SANS .json
            
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    lines = f.readlines()  # Lire le fichier ligne par ligne
                    
                    for line in lines:
                        # Extraire le mot après le 2ème ":"
                        parts = line.split(":")
                        if len(parts) > 2:
                            word_after_second_colon = parts[2].strip().split()[0]  # Prendre le premier mot après le deuxième ":"
                            
                            if word_after_second_colon in features:  # Vérifier si l'instruction est dans la liste
                                features[word_after_second_colon] = 1  # Marquer sa présence
                except Exception as e:
                    print(f"Erreur de lecture du fichier {filename} : {e}")

            data.append(features)

    # Convertir en DataFrame
    df = pd.DataFrame(data)
    
    # Sauvegarder en CSV
    df.to_csv(output_csv, index=False)
    
    print(f"DataFrame enregistré : {output_csv}")
    print(df.head())  # Afficher un aperçu

# Exemple d'utilisation
folder_path = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/folder_test_set"
instructions_list = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/instructions_list_cleaned.txt"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_test.csv"

create_features_dataframe(folder_path, instructions_list, output_csv)

étape_4_bis_fusion_test.py

In [ ]:
import pandas as pd

def merge_features_with_labels(features_csv, labels_csv, output_csv):
    # Charger les fichiers CSV
    df_features = pd.read_csv(features_csv)
    df_labels = pd.read_csv(labels_csv)  # Vérifier le bon séparateur
    
    # Renommer la colonne `filename` du fichier features pour qu'elle corresponde à `name`
    df_features.rename(columns={"filename": "name"}, inplace=True)

    # Fusionner les deux DataFrames sur `filename`
    df_final = df_labels.merge(df_features, on="name", how="inner")

    # Sauvegarder le DataFrame fusionné
    df_final.to_csv(output_csv, index=False)

    print(f"DataFrame fusionné enregistré : {output_csv}")
    print(df_final.head())  # Afficher un aperçu

features_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_test_inst.csv"
labels_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/test_data.csv"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/new_test_data.csv"

merge_features_with_labels(features_csv, labels_csv, output_csv)


étape_4_fusion_test.py

In [ ]:
import pandas as pd

def merge_features_with_labels(features_csv, labels_xlsx, output_csv):
    # Charger les fichiers CSV et Excel
    df_features = pd.read_csv(features_csv)
    df_labels = pd.read_excel(labels_xlsx)  # Lire le fichier Excel
    
    # Vérifier si la colonne `fikename` existe dans df_features
    if "filename" in df_features.columns:
        df_features.rename(columns={"filename": "name"}, inplace=True)
    else:
        print("Erreur : La colonne 'filename' est introuvable dans le fichier features.")
        return

    # Vérifier si la colonne `name` existe dans df_labels
    if "name" not in df_labels.columns:
        print("Erreur : La colonne 'name' est introuvable dans le fichier labels.")
        return

    # Fusionner les deux DataFrames sur `name`
    df_final = df_labels.merge(df_features, on="name", how="inner")

    # Sauvegarder le DataFrame fusionné
    df_final.to_csv(output_csv, index=False)

    print(f"DataFrame fusionné enregistré : {output_csv}")
    print(df_final.head())  # Afficher un aperçu

features_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/features_test.csv"
labels_xlsx = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/test_set_metadata_to_predict.xlsx"
output_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/test_data.csv"

merge_features_with_labels(features_csv, labels_xlsx, output_csv)


étape_5_prédiction.py

In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

# Fonction pour entraîner XGBoost
def train_xgboost(train_csv):
    # Charger le dataset
    df = pd.read_csv(train_csv)

    # Définir les features (X) et les labels (Y)
    label_columns = df.columns[4:454]  # Colonnes 5 à 454 = Labels
    feature_columns = df.columns[454:]  # Colonnes 455+ = Features extraites des instructions

    X = df[feature_columns].astype(np.float32)  # Convertir en float32 pour XGBoost
    Y = df[label_columns]  # Labels à prédire

    models = {}  # Stocker les modèles entraînés
    f1_scores = {}  # Stocker les scores

    # Séparer les données en train/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    for label in tqdm(Y.columns, desc="Training XGBoost Models"):
        print(f"Entraînement du modèle pour {label}...")

        # Création du modèle XGBoost
        model = xgb.XGBClassifier(
            objective="binary:logistic", eval_metric="logloss", 
            base_score=0.5  # Spécifier la valeur de base_score dans l'intervalle [0, 1]
        )

        # Entraînement
        model.fit(X_train, Y_train[label])

        # Prédiction et évaluation
        Y_pred = model.predict(X_test)
        f1 = f1_score(Y_test[label], Y_pred, average="macro")

        print(f"Macro F1 Score pour {label} : {f1:.4f}")
        models[label] = model
        f1_scores[label] = f1

    print("\nEntraînement terminé !")
    return models

# Fonction pour entraîner RandomForest
def train_random_forest(train_csv):
    # Charger le dataset
    df = pd.read_csv(train_csv)

    # Définir les features (X) et les labels (Y)
    label_columns = df.columns[4:454]  # Colonnes 5 à 454 = Labels
    feature_columns = df.columns[454:]  # Colonnes 455+ = Features extraites des instructions

    X = df[feature_columns].astype(np.float32)  # Convertir en float32 pour RandomForest
    Y = df[label_columns]  # Labels à prédire

    models = {}  # Stocker les modèles entraînés
    f1_scores = {}  # Stocker les scores

    # Séparer les données en train/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    for label in tqdm(Y.columns, desc="Training Random Forest Models"):
        print(f"Entraînement du modèle pour {label}...")

        # Création du modèle Random Forest
        model = RandomForestClassifier(n_estimators=100, random_state=42)

        # Entraînement
        model.fit(X_train, Y_train[label])

        # Prédiction et évaluation
        Y_pred = model.predict(X_test)
        f1 = f1_score(Y_test[label], Y_pred, average="macro")

        print(f"Macro F1 Score pour {label} : {f1:.4f}")
        models[label] = model
        f1_scores[label] = f1

    print("\nEntraînement terminé !")
    return models


def predict_test_data(models, test_csv, model_name):
    # Charger les données de test
    df_test = pd.read_csv(test_csv)

    # Définir les features utilisées (colonnes 455+)
    feature_columns = df_test.columns[454:]
    X_test = df_test[feature_columns].astype(np.float32)  # Convertir en float32

    # Prédire uniquement pour les labels ayant un modèle
    for label, model in tqdm(models.items(), desc=f"Predicting on {model_name} Test Data"):
        print(f"Prédiction pour {label}...")
        df_test[label] = model.predict(X_test).astype(int)  # Prédiction des valeurs

    # Sauvegarder les prédictions dans le même fichier
    df_test.to_csv(f"test_predictions_{model_name}.csv", index=False)

    print(f"\nPrédictions mises à jour dans : test_predictions_{model_name}.csv")


# Fichiers d'entrée
train_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/new_train_data.csv"
test_csv = "C:/Users/smohun/OneDrive - American International Group, Inc/Desktop/Data Challenge/new_test_data.csv"

# Entraînement et prédiction avec XGBoost
# xgboost_models = train_xgboost(train_csv)
# predict_test_data(xgboost_models, test_csv, "xgboost")

# Entraînement et prédiction avec RandomForest
rf_models = train_random_forest(train_csv)
predict_test_data(rf_models, test_csv, "rf")